tgb - 11/13/2019 - Testing if rescaling the inputs changes the ability of the network to generalize.   
Architecture = From [], predict []   
Test 1 = Rephrase all water concentrations as RH  
Test 2 = Rephrase all temperature as deviations from moist adiabat  
  
tgb - 11/15/2019 - This notebook is mostly used for pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# physical_devices = tf.config.experimental.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.experimental.set_memory_growth(physical_devices[1], True)
# tf.config.experimental.set_memory_growth(physical_devices[2], True)
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM



/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog
[Errno 2] No such file or directory: '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM'
/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# 1) Preprocessing

In [2]:
%cd /export/nfs0home/tbeucler/CBRAIN-CAM

/nfspool-0/home/tbeucler/CBRAIN-CAM


In [3]:
#!python preprocessing-11132019.py -c pp_config/2021_09_02_Small_data_test_For_Nando.yml

In [4]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Train_Valid_For_Nando.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/13/2021 03:57:13 AM Preprocess training dataset with real_geography flag set to False
09/13/2021 03:57:13 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc
09/13/2021 03:57:13 AM Reading input files
09/13/2021 03:57:13 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
09/13/2021 04:00:07 AM Crop levels
09/13/2021 04:00:07 AM Create stacked dataarray
09/13/2021 04:00:07 AM Real geography flag set to False
09/13/2021 04:00:07 AM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
09/13/2021 04:00:08 AM Stack and reshape dataarray
09/13/2021 04:00:38 AM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09

OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19318 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19457 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19458 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19460 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19459 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19461 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19462 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19463 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19464 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19465 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19466 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19467 thread 59 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19328 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19557 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19558 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19330 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19566 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19565 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19564 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19567 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19569 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19568 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19570 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19571 thread 158 bound to

OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19340 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19664 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19665 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19666 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19667 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19668 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19670 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19669 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19671 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19672 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19673 thread 259 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19675 thread 261 bound to 

OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19350 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19807 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19809 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19808 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19811 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19810 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19812 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19813 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19814 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19815 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19817 thread 359 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19816 thread 358 bound to 

OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19360 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19912 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19913 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19319 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20181 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20182 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20183 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20184 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20185 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20186 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20188 thread 458 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 20187 thread 457 bound 

09/13/2021 04:11:41 AM Done!
09/13/2021 04:11:41 AM Shuffle training dataset
09/13/2021 04:11:41 AM Start shuffling /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc into /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando_shuffle.nc. Open and create datasets.
09/13/2021 04:11:43 AM Shuffle!
100%|█████████████████████████████████████████████| 5/5 [01:45<00:00, 22.28s/it]
09/13/2021 04:13:29 AM Closing datasets
09/13/2021 04:13:29 AM Done!
09/13/2021 04:13:29 AM Preprocess validation dataset
09/13/2021 04:13:29 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_VALID_For_Nando.nc
09/13/2021 04:13:29 AM Reading input files
09/13/2021 04:13:29 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[5-8]-*-*.nc
09/13/2021 04:16:33 AM Crop levels
09/13/2021 04:16:33 AM Create stacked dataarray
09/13/2021 04:16:33 AM Real geography flag set t

OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19325 thread 648 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28587 thread 649 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28588 thread 650 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19353 thread 651 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28638 thread 653 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28636 thread 652 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 19345 thread 654 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28642 thread 655 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28643 thread 656 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28644 thread 657 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28645 thread 658 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 17829 tid 28646 thread 659 boun

In [5]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Test_for_Nando.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/13/2021 04:29:38 AM Preprocess training dataset with real_geography flag set to False
09/13/2021 04:29:38 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TEST_For_Nando.nc
09/13/2021 04:29:38 AM Reading input files
09/13/2021 04:29:38 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-0[6-9]-*-*.nc
09/13/2021 04:32:33 AM Crop levels
09/13/2021 04:32:33 AM Create stacked dataarray
09/13/2021 04:32:33 AM Real geography flag set to False
09/13/2021 04:32:33 AM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
09/13/2021 04:32:33 AM Stack and reshape dataarray
09/13/2021 04:33:05 AM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_

OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36209 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36307 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36308 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36310 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36309 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36311 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36313 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36314 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36312 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36315 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36317 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36316 thread 58 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36221 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36454 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36455 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36456 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36457 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36458 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36460 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36459 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36461 thread 158 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36464 thread 161 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36463 thread 160 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36462 thread 159 bound to 

OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36231 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36560 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36561 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36563 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36562 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36564 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36565 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36566 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36567 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36568 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36569 thread 259 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36570 thread 260 bound to 

OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36241 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36704 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36705 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36706 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36708 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36707 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36709 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36710 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36711 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36712 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36713 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36714 thread 359 bound to 

OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36251 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36809 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36810 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36210 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36812 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36813 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36814 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36815 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36816 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36817 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36819 thread 458 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 34355 tid 36818 thread 457 bound 

09/13/2021 04:45:01 AM Done!
09/13/2021 04:45:03 AM Shuffle training dataset
09/13/2021 04:45:03 AM Start shuffling /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TEST_For_Nando.nc into /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TEST_For_Nando_shuffle.nc. Open and create datasets.
09/13/2021 04:45:05 AM Shuffle!
100%|█████████████████████████████████████████████| 5/5 [01:35<00:00, 20.07s/it]
09/13/2021 04:46:41 AM Closing datasets
09/13/2021 04:46:41 AM Done!
09/13/2021 04:46:41 AM Finish entire preprocessing script.


In [6]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Valid_for_Nando_Cl_Inv.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/13/2021 04:46:52 AM Preprocess training dataset with real_geography flag set to False
09/13/2021 04:46:52 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_03_VALID_For_Nando_ClInv.nc
09/13/2021 04:46:52 AM Reading input files
09/13/2021 04:46:52 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[5-8]-*-*.nc
09/13/2021 04:47:04 AM Crop levels
09/13/2021 04:47:04 AM Create stacked dataarray
09/13/2021 04:47:04 AM Real geography flag set to False
09/13/2021 04:47:04 AM Aquaplanet simulation detected
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 packages x 12 c

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43864 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44214 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44215 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44216 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44217 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44218 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44219 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44220 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44221 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44224 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44222 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44223 thread 58 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43903 thread 168 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44335 thread 169 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44336 thread 170 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44337 thread 171 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44338 thread 172 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44339 thread 173 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44340 thread 174 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44341 thread 175 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44342 thread 176 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44344 thread 178 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44343 thread 177 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44345 thread 179 boun

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43872 thread 288 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44456 thread 290 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44455 thread 289 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44457 thread 291 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44458 thread 292 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44459 thread 293 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44460 thread 294 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44462 thread 296 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44461 thread 295 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44463 thread 297 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44464 thread 298 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44465 thread 299 bound to 

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43884 thread 408 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44571 thread 409 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44573 thread 411 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44572 thread 410 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44574 thread 412 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44575 thread 413 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44576 thread 414 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44577 thread 415 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44578 thread 416 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44579 thread 417 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44581 thread 419 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 44580 thread 418 boun

...

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43880 thread 648 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45008 thread 649 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45009 thread 650 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45011 thread 652 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45010 thread 651 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45012 thread 653 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45013 thread 654 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45014 thread 655 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45015 thread 656 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45017 thread 658 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45016 thread 657 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45028 thread 669 boun

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43866 thread 768 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45136 thread 769 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45137 thread 770 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45139 thread 772 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45140 thread 773 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45138 thread 771 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45142 thread 775 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45141 thread 774 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45143 thread 776 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45145 thread 778 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45144 thread 777 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45146 thread 779 bound to 

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43863 thread 888 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45433 thread 889 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45434 thread 890 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45435 thread 891 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45436 thread 892 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45437 thread 893 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45439 thread 895 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45438 thread 894 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45440 thread 896 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45441 thread 897 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45442 thread 898 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45443 thread 899 boun

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43861 thread 1008 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45609 thread 1009 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45610 thread 1010 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45611 thread 1011 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45612 thread 1012 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45613 thread 1013 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45615 thread 1015 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45618 thread 1018 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45614 thread 1014 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45617 thread 1017 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45616 thread 1016 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45619 thread 10

OMP: Info #250: KMP_AFFINITY: pid 43699 tid 43857 thread 1104 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45885 thread 1107 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45883 thread 1105 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45884 thread 1106 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45886 thread 1108 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45887 thread 1109 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45888 thread 1110 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45889 thread 1111 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45890 thread 1112 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45891 thread 1113 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45892 thread 1114 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 43699 tid 45893 thread 11

In [7]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Test_for_Nando_Cl_Inv.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/13/2021 05:18:16 AM Preprocess training dataset with real_geography flag set to False
09/13/2021 05:18:16 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_03_TEST_For_Nando_ClInv.nc
09/13/2021 05:18:16 AM Reading input files
09/13/2021 05:18:16 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-0[6-9]-*-*.nc
09/13/2021 05:18:29 AM Crop levels
09/13/2021 05:18:29 AM Create stacked dataarray
09/13/2021 05:18:29 AM Real geography flag set to False
09/13/2021 05:18:29 AM Aquaplanet simulation detected
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 packages x 12 co

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60276 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60602 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60603 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60604 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60605 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60606 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60607 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60608 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60610 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60611 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60609 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60612 thread 59 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60236 thread 168 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60723 thread 169 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60724 thread 170 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60725 thread 171 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60726 thread 172 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60727 thread 173 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60728 thread 174 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60729 thread 175 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60730 thread 176 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60732 thread 178 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60731 thread 177 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60733 thread 179 boun

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60232 thread 288 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60882 thread 289 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60883 thread 290 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60884 thread 291 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60885 thread 292 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60886 thread 293 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60887 thread 294 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60888 thread 295 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60889 thread 296 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60891 thread 298 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60890 thread 297 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60892 thread 299 bound to 

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60255 thread 408 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61000 thread 410 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60999 thread 409 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61001 thread 411 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61003 thread 413 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61002 thread 412 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61004 thread 414 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61005 thread 415 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61006 thread 416 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61008 thread 418 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61007 thread 417 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61009 thread 419 boun

...

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60250 thread 648 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61427 thread 649 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61428 thread 650 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61429 thread 651 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61432 thread 654 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61430 thread 652 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61431 thread 653 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61433 thread 655 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61434 thread 656 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61435 thread 657 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61436 thread 658 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61437 thread 659 boun

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60275 thread 768 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61554 thread 769 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61555 thread 770 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61556 thread 771 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61558 thread 773 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61557 thread 772 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61560 thread 775 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61559 thread 774 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61561 thread 776 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61562 thread 777 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61563 thread 778 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61564 thread 779 bound to 

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60235 thread 888 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61903 thread 889 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61904 thread 890 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61905 thread 891 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61906 thread 892 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61907 thread 893 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61908 thread 894 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61909 thread 895 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61910 thread 896 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61911 thread 897 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61912 thread 898 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 61913 thread 899 boun

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60272 thread 1008 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62308 thread 1009 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62309 thread 1010 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62310 thread 1011 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62311 thread 1012 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62312 thread 1013 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62313 thread 1014 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62314 thread 1015 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62315 thread 1016 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62318 thread 1019 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62316 thread 1017 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62317 thread 10

OMP: Info #250: KMP_AFFINITY: pid 60079 tid 60237 thread 1104 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62748 thread 1105 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62750 thread 1106 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62753 thread 1109 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62751 thread 1107 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62752 thread 1108 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62754 thread 1110 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62755 thread 1111 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62756 thread 1112 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62757 thread 1113 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62758 thread 1114 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 60079 tid 62759 thread 11

In [9]:
!python preprocessing-11132019.py -c pp_config/2021_09_13_Nando_ClInv_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/13/2021 06:11:30 AM Preprocess training dataset with real_geography flag set to False
09/13/2021 06:11:30 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_13_Nando_Cl_Inv_small.nc
09/13/2021 06:11:30 AM Reading input files
09/13/2021 06:11:30 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
09/13/2021 06:11:30 AM Crop levels
09/13/2021 06:11:30 AM Create stacked dataarray
09/13/2021 06:11:30 AM Real geography flag set to False
09/13/2021 06:11:30 AM Aquaplanet simulation detected
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 packages x 12

OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87565 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87625 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87624 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87626 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87627 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87628 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87629 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87630 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87631 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87632 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87633 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87634 thread 59 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87529 thread 168 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87740 thread 169 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87741 thread 170 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87742 thread 171 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87743 thread 172 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87744 thread 173 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87745 thread 174 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87747 thread 176 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87746 thread 175 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87748 thread 177 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87749 thread 178 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87750 thread 179 boun

OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87566 thread 288 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87861 thread 289 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87862 thread 290 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87863 thread 291 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87865 thread 293 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87866 thread 294 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87864 thread 292 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87868 thread 296 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87867 thread 295 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87871 thread 299 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87870 thread 298 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87869 thread 297 bound to 

OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87536 thread 408 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87982 thread 409 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87983 thread 410 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87984 thread 411 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87985 thread 412 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87986 thread 413 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87987 thread 414 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87988 thread 415 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87989 thread 416 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87990 thread 417 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87991 thread 418 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87992 thread 419 boun

OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87547 thread 552 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88126 thread 553 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88127 thread 554 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88128 thread 555 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88129 thread 556 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88130 thread 557 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88131 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88132 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88133 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88134 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88135 thread 562 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88136 thread 563 boun

OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87542 thread 666 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88281 thread 667 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88282 thread 668 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88284 thread 670 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88285 thread 671 bound to OS proc set 39
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88283 thread 669 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88286 thread 672 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88287 thread 673 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88288 thread 674 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88289 thread 675 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88290 thread 676 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88291 thread 677 bound to

OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87537 thread 761 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88379 thread 762 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88380 thread 763 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88381 thread 764 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88382 thread 765 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 87532 thread 766 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88385 thread 768 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88384 thread 767 bound to OS proc set 39
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88387 thread 770 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88386 thread 769 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88388 thread 771 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 87458 tid 88389 thread 772 bound t

In [8]:
!python preprocessing-11132019.py -c pp_config/2021_06_16_T_BMSE_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/13/2021 05:50:38 AM Preprocess training dataset with real_geography flag set to False
09/13/2021 05:50:38 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_16_BMSE_small.nc
09/13/2021 05:50:38 AM Reading input files
09/13/2021 05:50:38 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
09/13/2021 05:50:38 AM Crop levels
09/13/2021 05:50:38 AM Create stacked dataarray
09/13/2021 05:50:38 AM Real geography flag set to False
09/13/2021 05:50:38 AM Aquaplanet simulation detected
var is  QBP
var is  BMSE
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 package

OMP: Info #250: KMP_AFFINITY: pid 76853 tid 76974 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77069 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77070 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77071 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77072 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77073 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77074 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77075 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77076 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77078 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77079 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77077 thread 57 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 76853 tid 76937 thread 168 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77190 thread 169 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77191 thread 170 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77192 thread 171 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77193 thread 172 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77194 thread 173 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77195 thread 174 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77197 thread 176 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77196 thread 175 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77198 thread 177 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77199 thread 178 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77200 thread 179 boun

OMP: Info #250: KMP_AFFINITY: pid 76853 tid 76931 thread 288 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77306 thread 289 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77307 thread 290 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77308 thread 291 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77309 thread 292 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77310 thread 293 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77313 thread 296 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77311 thread 294 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77312 thread 295 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77315 thread 298 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77316 thread 299 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77314 thread 297 bound to 

var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: [47]
09/13/2021 05:50:48 AM Stack and reshape dataarray
09/13/2021 05:50:49 AM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_16_BMSE_small.nc
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 76962 thread 408 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77437 thread 409 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77438 thread 410 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77440 thread 412 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77439 thread 411 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77441 thread 413 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77442 thread 414 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77444 thread 416 bound to OS proc set 44
OMP: Info #250: KM

OMP: Info #250: KMP_AFFINITY: pid 76853 tid 76942 thread 506 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77530 thread 507 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77531 thread 508 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77532 thread 509 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77533 thread 510 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77534 thread 511 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77535 thread 512 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77536 thread 513 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77537 thread 514 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77538 thread 515 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77539 thread 516 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 76853 tid 77540 thread 517 boun

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_06_06_RG_Test-0K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
06/06/2021 10:28:44 AM Preprocess training dataset with real_geography flag set to True
06/06/2021 10:28:44 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_06_RG_TEST.nc
06/06/2021 10:28:44 AM Reading input files
06/06/2021 10:28:44 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_0001Kp_Neuralnet_SPCAM_all_216/atm/hist/SST_0001Kp_Neuralnet_SPCAM_all_216.cam.h1.2014-*-2*-*.nc
06/06/2021 10:37:38 AM Crop levels
06/06/2021 10:37:38 AM Create stacked dataarray
06/06/2021 10:37:38 AM Real geography flag set to True
06/06/2021 10:37:38 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINIT

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_06_03_RG_Train_Valid-0K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
06/04/2021 07:55:18 AM Preprocess training dataset with real_geography flag set to True
06/04/2021 07:55:18 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_03_RG_TRAIN.nc
06/04/2021 07:55:18 AM Reading input files
06/04/2021 07:55:18 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_0001Kp_Neuralnet_SPCAM_all_216/atm/hist/SST_0001Kp_Neuralnet_SPCAM_all_216.cam.h1.2013-*-0*-*.nc
06/04/2021 08:03:40 AM Crop levels
06/04/2021 08:03:40 AM Create stacked dataarray
06/04/2021 08:03:40 AM Real geography flag set to True
06/04/2021 08:03:40 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINI

In [18]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 01:07:56 AM Preprocess training dataset including output quantiles
04/25/2021 01:07:56 AM with real_geography flag set to True
04/25/2021 01:07:56 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TRAIN_M4K.nc
04/25/2021 01:07:56 AM Reading input files
04/25/2021 01:07:56 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.2009-*-0*-*.nc
04/25/2021 01:12:31 AM Reading PKL file containing univariate distribution of output variables
04/25/2021 01:12:31 AM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TRAIN_M4K_shuffle.nc_PERC.pkl
04/25/2021 01:12:31 AM Crop levels
04/25/2021 01:12:31 AM Create stacked dataarray
04/25/2021 01:12:31 AM Real geography flag set to True
04/25/2021 01:12:31 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLI

OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128624 thread 24 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128626 thread 25 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128589 thread 26 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128598 thread 27 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128616 thread 28 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128617 thread 29 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128584 thread 30 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128592 thread 31 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128591 thread 32 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128601 thread 33 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128599 thread 34 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128611 thread 35 bou

04/25/2021 08:10:34 AM Done!
04/25/2021 08:10:42 AM Finish entire preprocessing script.


In [19]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 08:16:12 AM Preprocess training dataset including output quantiles
04/25/2021 08:16:12 AM with real_geography flag set to True
04/25/2021 08:16:12 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TRAIN_P4K.nc
04/25/2021 08:16:12 AM Reading input files
04/25/2021 08:16:12 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.2009-*-0*-*.nc
04/25/2021 08:21:25 AM Reading PKL file containing univariate distribution of output variables
04/25/2021 08:21:25 AM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TRAIN_P4K_shuffle.nc_PERC.pkl
04/25/2021 08:21:25 AM Crop levels
04/25/2021 08:21:25 AM Create stacked dataarray
04/25/2021 08:21:25 AM Real geography flag set to True
04/25/2021 08:21:25 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var 

04/25/2021 04:21:17 PM Done!
04/25/2021 04:21:22 PM Finish entire preprocessing script.


In [20]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 04:26:51 PM Preprocess training dataset including output quantiles
04/25/2021 04:26:51 PM with real_geography flag set to True
04/25/2021 04:26:51 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TEST_P4K.nc
04/25/2021 04:26:51 PM Reading input files
04/25/2021 04:26:51 PM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.2010-*-2*-*.nc
04/25/2021 04:32:30 PM Reading PKL file containing univariate distribution of output variables
04/25/2021 04:32:30 PM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TEST_P4K.nc_PERC.pkl
04/25/2021 04:32:30 PM Crop levels
04/25/2021 04:32:30 PM Create stacked dataarray
04/25/2021 04:32:30 PM Real geography flag set to True
04/25/2021 04:32:30 PM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX


OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101687 thread 24 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101683 thread 25 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101660 thread 26 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101693 thread 27 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101662 thread 28 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101686 thread 29 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101653 thread 30 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101685 thread 31 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101690 thread 32 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101692 thread 33 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101672 thread 34 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101655 thread 35 bound to OS pro

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/26/2021 04:16:59 PM Preprocess training dataset including output quantiles
04/26/2021 04:16:59 PM with real_geography flag set to True
04/26/2021 04:16:59 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TEST_M4K.nc
04/26/2021 04:16:59 PM Reading input files
04/26/2021 04:16:59 PM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.2010-*-2*-*.nc
04/26/2021 04:21:47 PM Reading PKL file containing univariate distribution of output variables
04/26/2021 04:21:47 PM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TEST_M4K.nc_PERC.pkl
04/26/2021 04:21:47 PM Crop levels
04/26/2021 04:21:47 PM Create stacked dataarray
04/26/2021 04:21:47 PM Real geography flag set to True
04/26/2021 04:21:47 PM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_31_O3_T_NSto220_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_02_01_O3_BCONS_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_02_01_O3_QSATdeficit_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_input_small_data_test

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/118_POG_TfromTSinput_test.yml

In [ ]:
!ls -la -h /local/Tom.Beucler/SPCAM_PHYS/1*

# 2) Check the preprocessed data

In [8]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [8]:
path_RGsmall = path_data + '2021_04_18_NORM_RG_small.nc'

In [9]:
path_QSATdeficit = path_data+'2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [10]:
path_TfromNS = path_data+'2021_02_01_NORM_O3_TfromNS_small.nc'

In [9]:
path_Bcons = path_data+'2021_03_17_NORM_O3_BCONS_small.nc'

In [12]:
path_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'

In [13]:
data_RGsmall = xr.open_dataset(path_RGsmall)

In [13]:
# data_QSATdeficit = xr.open_dataset(path_QSATdeficit)
# data_TfromNS = xr.open_dataset(path_TfromNS)
# data_Bcons = xr.open_dataset(path_Bcons)
data_Bmse = xr.open_dataset(path_BMSE)

In [ ]:
data_QSATdeficit

In [ ]:
data_TfromNS

In [14]:
data_Bmse['var_names']

<xarray.DataArray 'var_names' (var_names: 184)>
array(['QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE',
       'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE',
       'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE',
       'BMSE', 'BMSE', 'BMSE', 'PS', 'SOLIN', 'SHFLX', 'LHFLX', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'QRL',
       'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL',
       'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL',
       'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRS',
       'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS',
       'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS',
       'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS'],
      dtype=object)
Coordinates:
  * var_names  (var_names) object 'QBP' 'QBP' 'QBP' 'QBP' ... 'QRS' 'QRS' 'QRS'

In [ ]:
data_TfromNS['mean'].shape

In [16]:
data_Bmse['mean'][:30].values

array([1.34937473e-06, 1.31343331e-06, 1.08532610e-06, 1.15688249e-06,
       1.13117293e-06, 1.11655880e-06, 1.13724015e-06, 1.21084406e-06,
       1.37536440e-06, 1.82399217e-06, 3.32074467e-06, 7.77086272e-06,
       1.85950039e-05, 4.11459191e-05, 8.63846857e-05, 1.69385693e-04,
       3.12721841e-04, 5.58001289e-04, 9.55813639e-04, 1.53020486e-03,
       2.36349273e-03, 3.60855338e-03, 4.97421819e-03, 5.76664708e-03,
       6.34710349e-03, 6.87083335e-03, 7.30065596e-03, 7.62099791e-03,
       7.80885332e-03, 8.26899336e-03])

In [17]:
data_Bmse['mean'][30:60].values

array([-4.00554868e+00, -5.51612582e+00, -7.53976408e+00, -4.92733378e+00,
       -4.75926980e+00, -4.06667155e+00, -3.29265373e+00, -2.76421626e+00,
       -2.39479296e+00, -2.03422430e+00, -1.67829473e+00, -1.32668592e+00,
       -1.00346602e+00, -7.32355679e-01, -5.21988785e-01, -3.62944323e-01,
       -2.42846319e-01, -1.49522538e-01, -7.19474030e-02, -7.33157626e-03,
        3.52665651e-02,  5.46795283e-02,  5.61516069e-02,  5.10403021e-02,
        4.21202158e-02,  3.24117022e-02,  2.38070651e-02,  1.66014297e-02,
        1.35225718e-02, -2.98660228e-04])

In [ ]:
plt.hist(data_Bmse['mean'])

In [ ]:
data_Bcons['mean'][30:60]

In [ ]:
data_Bcons['std'][30:60]

# 3) Preprocessing for PRL paper

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-RH-test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-test.yml